In [1]:
import cPickle
import os.path

api_key = cPickle.load( file( os.path.expanduser( '~/mediacloud_api_key.pickle' ), 'r' ) )

In [2]:
import cPickle
import os.path

cPickle.dump( api_key, file( os.path.expanduser( '~/mediacloud_api_key.pickle' ), 'wb' ) )

In [3]:
import mediacloud, json

mc = mediacloud.api.AdminMediaCloud( api_key)


# Flags

In [4]:
reload_stories = False
reprocess_with_cliff = True


In [5]:
controversy_stories = []

if reload_stories:
    
    last_processed_stories_id = 0 

    while True:
        print "fetching with last_processed_stories_id", last_processed_stories_id
        stories = mc.storyList( solr_query='{~ controversy:1113 }', rows=1000, corenlp=True, 
                               last_processed_stories_id=last_processed_stories_id)

        if len( stories ) == 0:
            break
            
        last_processed_stories_id = stories[-1]['processed_stories_id']

        controversy_stories.extend( stories)



    print len( controversy_stories)
    
    cPickle.dump( controversy_stories, 
                 file( os.path.expanduser( '~/Dropbox/gamergate_data/controversy_stories.pickle' ), 'wb' ) )

    #stories[0].keys()
    #[ s[ 'corenlp'] for s in stories ]

In [6]:
 controversy_stories = cPickle.load(file( os.path.expanduser( '~/Dropbox/gamergate_data/controversy_stories.pickle' ), 'r') )

In [7]:
def corenlp_annotated( s ):
    corenlp = s['corenlp']
    
    if 'annotated' not in corenlp:
        return True
    
    if corenlp['annotated'] == 'false':
        return False
    
    return True

In [8]:
corenlp_stories = [ s for s in controversy_stories if corenlp_annotated( s) ]

print len( controversy_stories )
len( corenlp_stories)
#corenlp_annotated( corenlp_stories[0] )

#corenlp_annotated( corenlp_stories[1] )
#[ s['corenlp'] for s in  corenlp_stories ]


16160


1889

In [10]:
#import cliff
from mediameter.cliff import Cliff

cliff_api = Cliff('http://civicdev.media.mit.edu', 8080)

In [11]:
if reprocess_with_cliff:
    for s in corenlp_stories:
        print s['stories_id']
        s['cliff_results'] = cliff_api.parseNlpJson( s['corenlp'])#['results']['people']

    cPickle.dump( corenlp_stories, 
                     file( os.path.expanduser( '~/Dropbox/gamergate_data/cliff_stories.pickle' ), 'wb' ) )


27208581
82202076
82689293
92198720
93258874
94482477
97982150
101622264
115021365
155643791
155875823
155998926
157321327
159036826
159301574
160768605
163402790
172602647
175718347
180892796
183617872
197232252
207900058
209597180
214846671
216937414
218070698
218069126
232899050
233568161
236344999
236754528
237175553
237807785
237822800
237825550
237893553
237919121
237959486
237976777
238016306
238101095
238167893
238200750
238256532
238414632
238466767
238476913
238476903
238497270
238522098
238665124
238719839
238832049
238834917
238837338
239046876
239083561
239169972
239241080
239423776
239515243
239644798
239919595
239952179
239961952
240015777
240036820
240047634
240069619
240072557
240130548
240351584
240404939
240400885
240456677
240471450
240509125
240568537
240657211
240653486
240725371
240736259
240910968
240937548
241010454
241154342
241163904
241185088
241316960
241329208
241329213
241398588
241407279
241447258
241649724
241660506
241681316
241692797
241695315
2417884

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [16]:
corenlp_stories = cPickle.load(file( os.path.expanduser( '~/Dropbox/gamergate_data/cliff_stories.pickle'), 'r') )

In [17]:
names_stories_map = {}

for s in corenlp_stories:
    
    if 'cliff_results' not in s:
        print 'skipping story', s['stories_id']
        continue
        
    results = s['cliff_results']['results']
    
    if 'people' not in results:
        continue
        
    people = results['people']
    names = [ p['name'] for p in people ]
    
    for name in names:
        if name not in names_stories_map:
            names_stories_map[ name ] = []
            
        names_stories_map[name].append( s )
        

len( names_stories_map.keys())


skipping story 281513910
skipping story 281615454
skipping story 281627205
skipping story 281690682
skipping story 281693175
skipping story 281869387
skipping story 281937114
skipping story 281953744
skipping story 282036123
skipping story 282033909
skipping story 282116593
skipping story 282295102
skipping story 282371284
skipping story 282383620
skipping story 282469141
skipping story 282473090
skipping story 282511210
skipping story 282628679
skipping story 282635918
skipping story 282734448
skipping story 282857595
skipping story 282939644
skipping story 282964111
skipping story 282972325
skipping story 282978356
skipping story 282984318
skipping story 283004895
skipping story 283078533
skipping story 283100082
skipping story 283130363
skipping story 283164207
skipping story 283173052
skipping story 283187674
skipping story 283187676
skipping story 283201628
skipping story 283196928
skipping story 283198147
skipping story 283207221
skipping story 283218383
skipping story 283231593


4009

In [34]:
from collections import Counter

c = Counter (  dict( [ ( name, len ( names_stories_map[ name ] ) ) for name in names_stories_map.keys() ]  ) )
             

del c['Sepp Blatter']
del c['Ray Rice']

c.most_common()


[(u'Anita Sarkeesian', 26),
 (u'Michel Platini', 16),
 (u'Greg Dyke', 14),
 (u'Zoe Quinn', 14),
 (u'Malky Mackay', 13),
 (u'Luis Suarez', 13),
 (u'Leigh Alexander', 12),
 (u'Iain Moody', 12),
 (u'Michael Garcia', 11),
 (u'Giorgio Chiellini', 10),
 (u'Messi', 9),
 (u'Harry Potter', 9),
 (u'Ronaldo', 9),
 (u'Donald Sterling', 8),
 (u'Kevin Pietersen', 8),
 (u'Gamasutra', 8),
 (u'Ian Botham', 7),
 (u'Eamon Dunphy', 7),
 (u'Emma Watson', 7),
 (u'Kim Kardashian', 7),
 (u'Richard Scudamore', 7),
 (u'Steve Ballmer', 7),
 (u'Jim Sterling', 7),
 (u'Mario', 7),
 (u'Lutyens', 6),
 (u'Nonglak Sahavattanapong', 6),
 (u'Forbes', 6),
 (u'Roger Goodell', 6),
 (u'Obama', 6),
 (u'Vincent ` Messi', 6),
 (u'Roy Hodgson', 6),
 (u'David', 6),
 (u'Doc Rivers', 6),
 (u'Ouseley', 5),
 (u'Jose Mourinho', 5),
 (u"Peter O ' Dwyer", 5),
 (u'Mohamed Bin Hammam', 5),
 (u'Max Payne', 5),
 (u'Lara Croft', 5),
 (u'John Smith', 5),
 (u'Fox', 5),
 (u'Jurgen Klinsmann', 5),
 (u'Dilma Rousseff', 5),
 (u'Vincent Tan', 5),
 